# Load Training Data

In [1]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
input_shape

30

In [4]:
train_names = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
train_column_names = train_names.values.tolist()
train_column_names = [k for row in train_column_names for k in row]

train_data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None)
train_data.columns = train_column_names

### Single dataframe column

y_train = pd.read_csv('../../../data/subject_train.txt', header = None)
y_train.columns = ['Subject']

y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
y_train_activity.columns = ['Activity']

X_train_1 = train_data[sub_features]
X_train_2 = train_data[act_features]
X_train_data = pd.concat([X_train_1, X_train_2], axis = 1)

X_train_data = pd.concat([X_train_data, y_train, y_train_activity], axis = 1)
X_train_data

,42 tGravityAcc-mean()-Y,43 tGravityAcc-mean()-Z,51 tGravityAcc-max()-Y,52 tGravityAcc-max()-Z,54 tGravityAcc-min()-Y,55 tGravityAcc-min()-Z,56 tGravityAcc-sma(),58 tGravityAcc-energy()-Y,"475 fBodyGyro-bandsEnergy()-1,8","483 fBodyGyro-bandsEnergy()-1,16",...,272 fBodyAcc-mad()-X,282 fBodyAcc-energy()-X,"303 fBodyAcc-bandsEnergy()-1,8","311 fBodyAcc-bandsEnergy()-1,16","315 fBodyAcc-bandsEnergy()-1,24",504 fBodyAccMag-std(),505 fBodyAccMag-mad(),509 fBodyAccMag-energy(),Subject,Activity
0,-0.140840,0.115375,-0.161265,0.124660,-0.123213,0.056483,-0.375426,-0.970905,-0.999454,-0.999619,...,-0.996889,-0.999968,-0.999963,-0.999969,-0.999971,-0.956134,-0.948870,-0.998285,1,5
1,-0.141551,0.109379,-0.161343,0.122586,-0.114893,0.102764,-0.383430,-0.970583,-0.999856,-0.999897,...,-0.997890,-0.999991,-0.999996,-0.999994,-0.999992,-0.975866,-0.975777,-0.999472,1,5
2,-0.142010,0.101884,-0.163711,0.094566,-0.114893,0.102764,-0.401602,-0.970368,-0.999954,-0.999962,...,-0.994097,-0.999969,-0.999989,-0.999983,-0.999972,-0.989015,-0.985594,-0.999807,1,5
3,-0.143976,0.099850,-0.163711,0.093425,-0.121336,0.095753,-0.400278,-0.969400,-0.999931,-0.999947,...,-0.994547,-0.999975,-0.999989,-0.999986,-0.999977,-0.986742,-0.983524,-0.999770,1,5
4,-0.148750,0.094486,-0.166786,0.091682,-0.121834,0.094059,-0.400477,-0.967051,-0.999926,-0.999946,...,-0.997725,-0.999990,-0.999994,-0.999993,-0.999991,-0.990063,-0.992324,-0.999873,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,-0.222004,-0.039492,-0.214233,-0.016391,-0.234998,-0.071977,-0.405132,-0.918375,-0.053258,-0.307101,...,-0.050748,-0.674230,-0.684177,-0.666429,-0.668164,-0.232600,-0.007392,-0.584282,30,2
7348,-0.242054,-0.039863,-0.231477,-0.016391,-0.234998,-0.068919,-0.358934,-0.902880,-0.029411,-0.286728,...,-0.177661,-0.705580,-0.726986,-0.704444,-0.705435,-0.275373,-0.172448,-0.632536,30,2
7349,-0.236950,-0.026805,-0.249134,0.024684,-0.216004,-0.068919,-0.377025,-0.907561,0.161404,-0.164197,...,-0.249486,-0.692379,-0.655263,-0.674515,-0.684729,-0.220288,-0.216074,-0.641170,30,2
7350,-0.233230,-0.004984,-0.244267,0.024684,-0.210542,-0.040009,-0.440050,-0.910648,0.193585,-0.155644,...,-0.247028,-0.693098,-0.643425,-0.677215,-0.685088,-0.234539,-0.220443,-0.663579,30,2


In [5]:
X_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,:-2].values
y_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,-2].values

In [6]:
for k in range(len(y_train)):
    if y_train[k] == 1:
        y_train[k] = 0
    elif y_train[k] == 3:
        y_train[k] = 1
    elif y_train[k] == 5:
        y_train[k] = 2
    elif y_train[k] == 7:
        y_train[k] = 3
    else:
        y_train[k] = 4

# Train Test Data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.15, shuffle = True)

In [8]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 20),
            classifier_block(20, 20),
            classifier_block(20, 10),
            nn.Linear(10, 5)
        )
    def forward(self, x):
        return self.network(x)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Classifier()
lr = 0.001
n_epochs = 7500
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [10]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.901568055152893, Final Batch Loss: 1.6216896772384644
Epoch 2, Loss: 4.9057323932647705, Final Batch Loss: 1.6585233211517334
Epoch 3, Loss: 4.889178991317749, Final Batch Loss: 1.6292177438735962
Epoch 4, Loss: 4.882572889328003, Final Batch Loss: 1.6380025148391724
Epoch 5, Loss: 4.871217489242554, Final Batch Loss: 1.6309311389923096
Epoch 6, Loss: 4.862881660461426, Final Batch Loss: 1.602850079536438
Epoch 7, Loss: 4.8588597774505615, Final Batch Loss: 1.623079776763916
Epoch 8, Loss: 4.847156524658203, Final Batch Loss: 1.5948899984359741
Epoch 9, Loss: 4.859623551368713, Final Batch Loss: 1.627223253250122
Epoch 10, Loss: 4.844614386558533, Final Batch Loss: 1.6101192235946655
Epoch 11, Loss: 4.843320965766907, Final Batch Loss: 1.6028560400009155
Epoch 12, Loss: 4.8378989696502686, Final Batch Loss: 1.6072643995285034
Epoch 13, Loss: 4.841763377189636, Final Batch Loss: 1.6190848350524902
Epoch 14, Loss: 4.840689182281494, Final Batch Loss: 1.6151691675186157
E

Epoch 117, Loss: 2.682243525981903, Final Batch Loss: 0.8801479935646057
Epoch 118, Loss: 2.699390709400177, Final Batch Loss: 0.9007763266563416
Epoch 119, Loss: 2.642258882522583, Final Batch Loss: 0.9156909584999084
Epoch 120, Loss: 2.732376754283905, Final Batch Loss: 0.9401359558105469
Epoch 121, Loss: 2.6884477734565735, Final Batch Loss: 0.9654163718223572
Epoch 122, Loss: 2.6603047847747803, Final Batch Loss: 0.8705951571464539
Epoch 123, Loss: 2.6441264152526855, Final Batch Loss: 0.8912725448608398
Epoch 124, Loss: 2.551886796951294, Final Batch Loss: 0.8756913542747498
Epoch 125, Loss: 2.5871490240097046, Final Batch Loss: 0.8637918829917908
Epoch 126, Loss: 2.64469051361084, Final Batch Loss: 0.917938232421875
Epoch 127, Loss: 2.513135612010956, Final Batch Loss: 0.8211983442306519
Epoch 128, Loss: 2.6586586236953735, Final Batch Loss: 0.9673818349838257
Epoch 129, Loss: 2.58806836605072, Final Batch Loss: 0.8297412991523743
Epoch 130, Loss: 2.4445075392723083, Final Batch 

Epoch 238, Loss: 1.802367925643921, Final Batch Loss: 0.6153924465179443
Epoch 239, Loss: 1.8050596714019775, Final Batch Loss: 0.6184664368629456
Epoch 240, Loss: 1.6862937808036804, Final Batch Loss: 0.5218042731285095
Epoch 241, Loss: 1.7541447281837463, Final Batch Loss: 0.5904802083969116
Epoch 242, Loss: 1.8352892994880676, Final Batch Loss: 0.5614187121391296
Epoch 243, Loss: 1.723652720451355, Final Batch Loss: 0.5710246562957764
Epoch 244, Loss: 1.7119999527931213, Final Batch Loss: 0.5188656449317932
Epoch 245, Loss: 1.7001781165599823, Final Batch Loss: 0.48970451951026917
Epoch 246, Loss: 1.6680594086647034, Final Batch Loss: 0.5188875198364258
Epoch 247, Loss: 1.6027894020080566, Final Batch Loss: 0.525425374507904
Epoch 248, Loss: 1.690609872341156, Final Batch Loss: 0.5018102526664734
Epoch 249, Loss: 1.7736746072769165, Final Batch Loss: 0.6271088719367981
Epoch 250, Loss: 1.7160799503326416, Final Batch Loss: 0.6262131929397583
Epoch 251, Loss: 1.665577471256256, Final

Epoch 350, Loss: 1.4373565018177032, Final Batch Loss: 0.44295641779899597
Epoch 351, Loss: 1.398557186126709, Final Batch Loss: 0.4735576808452606
Epoch 352, Loss: 1.3773577213287354, Final Batch Loss: 0.44377079606056213
Epoch 353, Loss: 1.4509322345256805, Final Batch Loss: 0.4746485650539398
Epoch 354, Loss: 1.490724802017212, Final Batch Loss: 0.4596511721611023
Epoch 355, Loss: 1.4248591661453247, Final Batch Loss: 0.4229174554347992
Epoch 356, Loss: 1.5103453695774078, Final Batch Loss: 0.484640896320343
Epoch 357, Loss: 1.4165081679821014, Final Batch Loss: 0.41348695755004883
Epoch 358, Loss: 1.4484745562076569, Final Batch Loss: 0.5045866966247559
Epoch 359, Loss: 1.4773278832435608, Final Batch Loss: 0.5152537822723389
Epoch 360, Loss: 1.4410142600536346, Final Batch Loss: 0.4688650071620941
Epoch 361, Loss: 1.410400778055191, Final Batch Loss: 0.4825444221496582
Epoch 362, Loss: 1.5225830972194672, Final Batch Loss: 0.5209389328956604
Epoch 363, Loss: 1.3476468920707703, Fi

Epoch 470, Loss: 1.268854945898056, Final Batch Loss: 0.4094243347644806
Epoch 471, Loss: 1.3693192303180695, Final Batch Loss: 0.44996702671051025
Epoch 472, Loss: 1.3348695635795593, Final Batch Loss: 0.4116398096084595
Epoch 473, Loss: 1.2391521334648132, Final Batch Loss: 0.46619072556495667
Epoch 474, Loss: 1.2727529406547546, Final Batch Loss: 0.48455968499183655
Epoch 475, Loss: 1.3472613394260406, Final Batch Loss: 0.47556746006011963
Epoch 476, Loss: 1.343614012002945, Final Batch Loss: 0.4376334547996521
Epoch 477, Loss: 1.2374318838119507, Final Batch Loss: 0.4100606143474579
Epoch 478, Loss: 1.2849087119102478, Final Batch Loss: 0.3456926643848419
Epoch 479, Loss: 1.2785455584526062, Final Batch Loss: 0.4392121136188507
Epoch 480, Loss: 1.3450663685798645, Final Batch Loss: 0.47575846314430237
Epoch 481, Loss: 1.3041541874408722, Final Batch Loss: 0.45426735281944275
Epoch 482, Loss: 1.3555095195770264, Final Batch Loss: 0.5252493023872375
Epoch 483, Loss: 1.229950100183487

...

Epoch 6973, Loss: 0.2584000304341316, Final Batch Loss: 0.08933430910110474
Epoch 6974, Loss: 0.24147772043943405, Final Batch Loss: 0.08978106826543808
Epoch 6975, Loss: 0.35230913758277893, Final Batch Loss: 0.10763660073280334
Epoch 6976, Loss: 0.2677266523241997, Final Batch Loss: 0.07106894254684448
Epoch 6977, Loss: 0.3199850246310234, Final Batch Loss: 0.10822685807943344
Epoch 6978, Loss: 0.2554694600403309, Final Batch Loss: 0.05695933476090431
Epoch 6979, Loss: 0.3246418237686157, Final Batch Loss: 0.12495245784521103
Epoch 6980, Loss: 0.3138650432229042, Final Batch Loss: 0.06670712679624557
Epoch 6981, Loss: 0.31256283819675446, Final Batch Loss: 0.16391126811504364
Epoch 6982, Loss: 0.43729446828365326, Final Batch Loss: 0.12060213088989258
Epoch 6983, Loss: 0.2730840928852558, Final Batch Loss: 0.06093503162264824
Epoch 6984, Loss: 0.274670273065567, Final Batch Loss: 0.11426834017038345
Epoch 6985, Loss: 0.29099805653095245, Final Batch Loss: 0.12252725660800934
Epoch 69

Epoch 7090, Loss: 0.36189092695713043, Final Batch Loss: 0.10189060121774673
Epoch 7091, Loss: 0.32001592218875885, Final Batch Loss: 0.07159650325775146
Epoch 7092, Loss: 0.31908831745386124, Final Batch Loss: 0.0867721289396286
Epoch 7093, Loss: 0.30858440697193146, Final Batch Loss: 0.1033484935760498
Epoch 7094, Loss: 0.37321067601442337, Final Batch Loss: 0.14911717176437378
Epoch 7095, Loss: 0.32834333926439285, Final Batch Loss: 0.11515506356954575
Epoch 7096, Loss: 0.4616669863462448, Final Batch Loss: 0.15830326080322266
Epoch 7097, Loss: 0.31427448242902756, Final Batch Loss: 0.09247704595327377
Epoch 7098, Loss: 0.4446321874856949, Final Batch Loss: 0.23393894731998444
Epoch 7099, Loss: 0.3408070430159569, Final Batch Loss: 0.08769595623016357
Epoch 7100, Loss: 0.2752075716853142, Final Batch Loss: 0.07476002722978592
Epoch 7101, Loss: 0.35287298262119293, Final Batch Loss: 0.11102461069822311
Epoch 7102, Loss: 0.36678793281316757, Final Batch Loss: 0.13655106723308563
Epoch

Epoch 7205, Loss: 0.23967793211340904, Final Batch Loss: 0.13132880628108978
Epoch 7206, Loss: 0.3235303983092308, Final Batch Loss: 0.11310181766748428
Epoch 7207, Loss: 0.41928109526634216, Final Batch Loss: 0.13123001158237457
Epoch 7208, Loss: 0.3612795025110245, Final Batch Loss: 0.15519261360168457
Epoch 7209, Loss: 0.42688556760549545, Final Batch Loss: 0.12075062841176987
Epoch 7210, Loss: 0.2711128294467926, Final Batch Loss: 0.07799432426691055
Epoch 7211, Loss: 0.3950212523341179, Final Batch Loss: 0.16934101283550262
Epoch 7212, Loss: 0.3191375955939293, Final Batch Loss: 0.07157079130411148
Epoch 7213, Loss: 0.2896561175584793, Final Batch Loss: 0.08620275557041168
Epoch 7214, Loss: 0.3594762086868286, Final Batch Loss: 0.13504095375537872
Epoch 7215, Loss: 0.3152570500969887, Final Batch Loss: 0.08160229027271271
Epoch 7216, Loss: 0.27646975964307785, Final Batch Loss: 0.10495895147323608
Epoch 7217, Loss: 0.35636525601148605, Final Batch Loss: 0.08000615239143372
Epoch 7

Epoch 7321, Loss: 0.34024207293987274, Final Batch Loss: 0.08649294823408127
Epoch 7322, Loss: 0.3114653378725052, Final Batch Loss: 0.1099485531449318
Epoch 7323, Loss: 0.39929115027189255, Final Batch Loss: 0.08841542154550552
Epoch 7324, Loss: 0.32475724816322327, Final Batch Loss: 0.09589996188879013
Epoch 7325, Loss: 0.24248458817601204, Final Batch Loss: 0.03892165794968605
Epoch 7326, Loss: 0.30547139048576355, Final Batch Loss: 0.11467929929494858
Epoch 7327, Loss: 0.26603494584560394, Final Batch Loss: 0.0723535344004631
Epoch 7328, Loss: 0.28086449950933456, Final Batch Loss: 0.07623746246099472
Epoch 7329, Loss: 0.2750852033495903, Final Batch Loss: 0.05061531066894531
Epoch 7330, Loss: 0.33668647706508636, Final Batch Loss: 0.12161947786808014
Epoch 7331, Loss: 0.26784225553274155, Final Batch Loss: 0.0920214056968689
Epoch 7332, Loss: 0.2834034562110901, Final Batch Loss: 0.07728181779384613
Epoch 7333, Loss: 0.3217248246073723, Final Batch Loss: 0.14903464913368225
Epoch 

Epoch 7434, Loss: 0.2830953896045685, Final Batch Loss: 0.10886414349079132
Epoch 7435, Loss: 0.2774876207113266, Final Batch Loss: 0.09843682497739792
Epoch 7436, Loss: 0.3221694529056549, Final Batch Loss: 0.1002642884850502
Epoch 7437, Loss: 0.24313290417194366, Final Batch Loss: 0.05048880726099014
Epoch 7438, Loss: 0.3074406534433365, Final Batch Loss: 0.10516322404146194
Epoch 7439, Loss: 0.31838933378458023, Final Batch Loss: 0.12998926639556885
Epoch 7440, Loss: 0.4217112511396408, Final Batch Loss: 0.1342920958995819
Epoch 7441, Loss: 0.24615608528256416, Final Batch Loss: 0.05086180195212364
Epoch 7442, Loss: 0.32266274094581604, Final Batch Loss: 0.14218147099018097
Epoch 7443, Loss: 0.28768355771899223, Final Batch Loss: 0.056799713522195816
Epoch 7444, Loss: 0.34042640775442123, Final Batch Loss: 0.10658755898475647
Epoch 7445, Loss: 0.22563040629029274, Final Batch Loss: 0.05184543505311012
Epoch 7446, Loss: 0.3621576875448227, Final Batch Loss: 0.16445042192935944
Epoch 

In [24]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[24  0  0  0  0]
 [ 0 27  0  0  0]
 [ 0  0 26  1  0]
 [ 0  0  2 20  0]
 [ 0  1  1  0 16]]
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        24
           1      0.964     1.000     0.982        27
           2      0.897     0.963     0.929        27
           3      0.952     0.909     0.930        22
           4      1.000     0.889     0.941        18

    accuracy                          0.958       118
   macro avg      0.963     0.952     0.956       118
weighted avg      0.959     0.958     0.958       118



In [25]:
torch.save(model.state_dict(), f'../../../saved_models/UCI 5 User Classifier Ablation')